# Designing recursive functions

Now that we have seen how recursion works, we will focus on how to design recursive functions starting from an analysis of a problem. We will then translate the recursive functions in Python.

## General strategy
When confronted with a difficult problem, it is always helpful to draw it. The ability to draw something out is not just useful for aesthetic reasons: drawing the problem (with precision!) forces us to reckon with all the details of the problem, and to visualise them so we can properly deal with them.

After drawing or plotting the problem, we look for _regularities_. Regularities are powerful elements in a solution, because they suggest the building blocks of both the problem and therefore the code that will become the solution. Such regularities present themselves as smaller instances of the problem itself, which we also call _recurring structure_ or _inductive case_. Without regularities (or without seeing them) it is very hard to produce a solution.

After we have found the recurring structure, we must find what ties them together. For example, even though it is true that a line is made up of smaller lines, we must also define what "made up of" really means: in this case, concatenation.

The recurring structure should recur in a specific way: every time, at a smaller scale. The line must be made up of _smaller_ lines, and at some point we expect a line so small that we do not really need to split it further. This definition of when do we stop splitting our problem into smaller versions of itself is called the _base case_.

## Playing with numbers

As a first example, let us consider the problem of computing the sum of all numbers in the range $0 \dots n$ for a given $n$. The recurring structure can be seen just by putting brackets to identify that a sum is actually a smaller sum, plus the last number:

$$(0 + 1 + 2 + \dots + n-1 + n) = ((((0 + 1) + 2)\dots) + n-1) + n$$

The brackets explicitly denote a sub-problem. Of course, we could rephrase this in a way that makes it even more evident where the recurring structure occurs. Whenever we have a series of numbers, such as $0 + \dots + n$ added together and within brackets, we will write $\text{sum}(0,n)$:

\begin{align*}
\text{sum}(0,n) 
&= \text{sum}(0,n-1) + n \\
&= (\text{sum}(0,n-2) + n-1) + n \\
&= ((\text{sum}(0,n-3) + n-2) + n-1) + n \\
&= \vdots \\
&= (((0 + 1 \dots) + n-2) + n-1) + n \\
\end{align*}

This formulation should also make it evident how the base case will be zero: adding all numbers up to zero will simply result in zero itself.

We can translate this into a recursive function as follows:

In [6]:
def sum(n):
    if n <= 0: return 0
    else: return sum(n-1) + n
print(sum(int(input())))

6
21


We could of course also look for other recurring structures. For example, we could notice that the sum of numbers up until $n$ is actually the sum of all numbers within the range $0 \dots n$. If we focus on the sum over a range, then more decomposition possibilities emerge. One of these would, for example, split the range in two half ranges:

$$0 + 1 + \dots + n = (0 + 1 + \dots + n//2) + ((n//2+1) + \dots + n)$$

For example, the sum $0 + 1 + 2 + 3 + 4 + 5$ could be reformulated as the sum of $0 + 1 + 2$ and $3 + 4 + 5$. The sub-sequences respectively end at the middle point of the previous subsequence, $2 = 5//2$, and start at its successor $3 = (5//2 + 1)$.

Until now, the only variable in this formulation was the last number to add, $n$. To add an extra step of complexity in this formulation, we could also vary the beginning of the range, which until now was $0$. We will denote the beginning of the sequence with $l$ (_lower_ bound) and the end of the sequence with $u$ (_upper_ bound).

When adding (sub-) sequences, we actually have two base cases: the empty sequence, when $l > u$ (it is empty because there can be no values between numbers where the first is greater than the second), adds to $0$; the sequence with only one element, when $l = u$, adds to $l$.

\begin{align*}
\text{sum_range}(l,u) &= 0 \text{ when } l > u \\
&= l \text{ when } l = u \\
&= \text{sum_range}(l,m) + \text{sum_range}(m+1,u) \text{ otherwise, where } m = (l+u)//2 \\
\end{align*}

Turned into code, this becomes:

In [15]:
def sum_range(l, u):
    if l > u:
        return 0
    elif l == u:
        return l
    else:
        middle_point = (l + u) // 2
        fst_half = sum_range(l, middle_point)
        snd_half = sum_range(middle_point + 1, u)
        return fst_half + snd_half

def sum(n):
    return sum_range(1, n)

print(sum(int(input())))

7
28


### More examples with numbers

We can perform a very similar operation to adding a range of numbers: multiplying a range. For example, the product of numbers up until $3$ would be $1 \times 2 \times 3$.

Just like we did for the sum, we can decompose the product into the product of a shorter range, multiplied by the last number. Thus, the product of an empty sequence of numbers is $\text{prod}(0) = 1$ (should we choose $0$, then we would always get $0$ also for non-empty sequences!), whereas the product of a non-empty sequence of numbers up to $n$ is the product of the sequence up to $n-1$, multiplied by $n$: $\text{prod}(n) = \text{prod}(n-1) \times n$.

This becomes the simple recursive function:

In [23]:
def prod(n):
    if n <= 0:
        return 1
    else:
        return prod(n-1) * n
print(prod(int(input())))

4
24


With the very same trick, we could define the power $b^a = \underbrace{b \times \dots \times b}_{a \text{ times}}$ as $1$ when $a=0$, but $\underbrace{b \times \dots \times b}_{a-1 \text{ times}} \times b$ otherwise.

This directly translates into the (now familiar-looking) code:

In [26]:
def pow(b,a):
    if a <= 0:
        return 1
    else:
        return pow(b,a-1) * b
print(pow(int(input()), int(input())))

4
2
16


## Drawing recursively

Let us now go back to defining functions to draw pictures, this time using recursion in order to decompose shapes into smaller shapes of the same sort.

### Rectangles
Let us consider a rectangle. The decomposition we have made so far was based on the idea that a (non-empty) rectangle is indeed just the first row, followed by a rectangle with one less row.

<img src="images/rectangle_to_row_rectangle.png" alt="Rectangle to row and rectangle" style="width: 400px;"/>

This decomposition means therefore that drawing a rectangle recursively will first check whether or not we are dealing with a (degenerate) rectangle with no rows. If that is the case, then we are done and we can return the base case, which will be an empty string. If the rectangle has indeed any rows to draw, we draw the first one as a line, and then the remaining rows by simply recalling drawing a smaller rectangle:

In [2]:
def line(n): return "" if n <= 0 else "*" + line(n-1)
    
def rectangle(n,m):
    if n <= 0: 
        return ""
    else:
        l = line(m)
        r = rectangle(n-1,m)
        return l + "\n" + r
    
n = int(input())
print(rectangle(n,n))

4
****
****
****
****



We could also observe that a rectangle is not necessarily decomposed as a row, then another rectangle. We could also see a rectangle as just the sum of two smaller rectangles, both roughly half the number of rows of the original. 

<img src="images/rectangle_to_rectangles.png" alt="Rectangle to two half rectangles" style="width: 600px;"/>

Of course, care must be taken when splitting in two: we must, at some point, draw the actual rows, otherwise we will end with no drawing at all. For this reason, the following implementation would not draw anything, because it would keep splitting until eventually `n` would become smaller than one and then only the empty string would be returned:

```
rectangle := (n,m) =>
  if n <= 0 then 
    return ""
  else
    return rectangle(n//2,m) + "\n" + rectangle(n//2,m)
  
```

To solve this, we must make sure that we never split a rectangle with only one row. This leads us to two base cases: one for `n = 0`, but also one for `n = 1`:

In [7]:
def line(n): return "" if n <= 0 else "*" + line(n-1)

def rectangle(n,m):
    if n <= 0:
        return ""
    elif n == 1:
        return line(m) + "\n"
    else:
        return rectangle(n//2,m) + rectangle(n-n//2,m)

n = int(input())
print(rectangle(n,n))

6
******
******
******
******
******
******



Also notice that the first sub-rectangle has `n//2` rows, but the second sub-rectangle has `n - n//2` rows. The reason why this is useful is that we know that the two subrectangles together must have exactly `n` rows. We could have used `n//2` for both subrectangles, but this would have led us to issues when `n` is not even. In that case we would need yet another check to make sure that if `n` is odd, then we somehow add an extra row to one of the rectangles, thereby complicating our code needlessly. 

The solution proposed in the code above on the other hand, is much simpler. We simply state that the first rectangle will have `n//2` rows, which is reasonable. Then, we state that the second rectangle will have _the remaining rows_. There is no reason to define constructs based on complex reasoning when a much simpler variation ensures the same results without extra effort.

### Triangles
The exact same reasoning can be applied to drawing triangles, but this time with a minor twist. Suppose we wanted to decompose a triangle by isolating one row per step: depending on which row we remove, the rest of the triangle might stop being a triangle!

<img src="images/triangle_to_row_triangle.png" alt="Triangle to row and triangle, both ways" style="width: 600px;"/>

This means that we are actually forced to decompose the last row away, otherwise we will not be able to re-use the triangle function recursively:

In [17]:
def line(n): return "" if n <= 0 else "*" + line(n-1)

def triangle(n):
    if n <= 0:
        return ""
    else:
        return triangle(n-1) + line(n) + "\n"
print(triangle(int(input())))

5
*
**
***
****
*****



Suppose on the other hand that, infatuated with the beauty of recursion and the idea of splitting the problem in a balanced way (that is two halves per step), we want to follow the `n//2` strategy for triangles as well.

<img src="images/trapeze_to_trapezes.png" alt="Trapeze to trapezes" style="width: 600px;"/>

It is clear that the splitting always produces two subfigures which are not triangles. Specifically, one of them is called a _trapeze_ (or _trapezoid_). Fortunately, we can see that the upper half (which still looks like a triangle) is a trapeze as well, just a degenerate one in which the upper side is "collapsed" to a point.

Armed with this realisation, we can simply define the `trapeze(u,l)` function which draws a trapeze with an upper side of length `u` and a lower side of length `l`:

In [3]:
def line(n): return "" if n <= 0 else "*" + line(n-1)

def trapeze(u,l):
    if l < u:
        return ""
    elif l == u:
        return line(u) + "\n"
    else:
        m = (u + l) // 2
        return trapeze(u,m) + trapeze(m+1,l)
    
def triangle(n):
    return trapeze(1,n)

print(triangle(int(input())))

10
*
**
***
****
*****
******
*******
********
*********
**********

